In [203]:
!python -m pip install unidic-lite
!python -m pip install fugashi
!echo "y" | conda install pytorch torchvision torchaudio cudatoolkit=11.6 -c pytorch -c conda-forge
!python -m pip install transformers
!python -m pip install ir_datasets
!echo "y" | conda install -c huggingface -c conda-forge datasets
!python -m pip install -U scikit-learn

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 22.9.0

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.3
  latest version: 22.9.0

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/argon/anaconda3/envs/jpir

  added / updated specs:
    - datasets


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    datasets-2.6.1             |             py_0         316 KB  huggingface
    ------------------------------------------------------------
                                           Total:         316 KB

The following packages will be UPDATED:

  datasets            

In [1]:
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM, BertModel
model_name_or_path = "cl-tohoku/bert-base-japanese-v2"

/home/argon/anaconda3/envs/jpir/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name_or_path = "cl-tohoku/bert-base-japanese-v2"

In [3]:
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
model = BertForMaskedLM.from_pretrained(model_name_or_path)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
input_ids = tokenizer.encode(f"青葉山で{tokenizer.mask_token}の研究をしています。", return_tensors="pt")
print(input_ids)
print(tokenizer.convert_ids_to_tokens(input_ids[0].tolist()))
masked_index = torch.where(input_ids == tokenizer.mask_token_id)[1][0].tolist()
print(masked_index)


tensor([[    2, 21479,  2077,   889,     4,   896, 11261,   932,   873,   888,
           854, 12343,   829,     3]])
['[CLS]', '青葉', '山', 'で', '[MASK]', 'の', '研究', 'を', 'し', 'て', 'い', 'ます', '。', '[SEP]']
4


In [14]:
result = model(input_ids)
pred_ids = result[0][:, masked_index].topk(5).indices.tolist()[0]
print(result.logits.shape)
for pred_id in pred_ids:
    output_ids = input_ids.tolist()[0]
    output_ids[masked_index] = pred_id
    print(tokenizer.decode(output_ids))

torch.Size([1, 14, 32768])
[CLS] 青葉 山 で 植物 の 研究 を し て い ます 。 [SEP]
[CLS] 青葉 山 で 鳥類 の 研究 を し て い ます 。 [SEP]
[CLS] 青葉 山 で 野鳥 の 研究 を し て い ます 。 [SEP]
[CLS] 青葉 山 で 恐竜 の 研究 を し て い ます 。 [SEP]
[CLS] 青葉 山 で 昆虫 の 研究 を し て い ます 。 [SEP]


In [15]:
from datasets import load_dataset

dataset = load_dataset('unicamp-dl/mmarco', 'japanese')


Found cached dataset mmarco (/home/argon/.cache/huggingface/datasets/unicamp-dl___mmarco/japanese/2.0.0/38e5159e2b13a7fdcd2af6415542b3c9a3bf27d70031abd9c4f814a2aa3367e2)
100%|██████████| 1/1 [00:03<00:00,  3.68s/it]


In [78]:
model = BertModel.from_pretrained(model_name_or_path)
tokenizer.add_tokens("[Q]", special_tokens=True)
model.resize_token_embeddings(len(tokenizer))

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Embedding(32769, 768)

In [90]:
first_data = dataset["train"][0]
print(len(tokenizer))
first_data

32769


{'query': '妊娠中は少しカフェインで大丈夫です',
 'positive': '私たちはあなたとあなたの赤ちゃんに対する妊娠中のカフェインの影響について多くを知りません。したがって、毎日取得する量を制限するのが最善です。妊娠している場合は、カフェインを1日200ミリグラムに制限してください。これは、1×8オンスのコーヒー1杯または1杯の12オンスのコーヒーの量とほぼ同じです。',
 'negative': '妊娠中にチョコレートを食べることの特定の利点が研究によって証明されているため、妊娠中の女性がチョコレートを食べることは一般的に安全です。ただし、妊娠中の女性は、カフェインの摂取量が1日あたり200mg未満であることを確認する必要があります。'}

In [89]:
# tokenizer(["[Q]"+first_data["query"], "[Q]second query"], return_tensors="pt")
# model.logits.shape
texts = [
    "[Q]first query",
    "[Q]second query",
]
inputs = tokenizer(texts, return_tensors="pt",padding=True, truncation=True, max_length=512)
print(inputs.input_ids, ":", inputs.input_ids.shape)
print(model(**inputs).last_hidden_state[:, 0].shape)
model(**inputs).last_hidden_state.shape

tensor([[    2, 32768,    84, 27291, 27647, 25821,     3,     0],
        [    2, 32768, 22537, 30124,  6242, 27647, 25821,     3]]) : torch.Size([2, 8])
torch.Size([2, 768])


torch.Size([2, 8, 768])

In [65]:
tokenizer.decode([32768]) 

'[Q]'

In [110]:
# todo
# - スコア計算のテスト

In [169]:
query_tokenized = mmarco_collator(dataset["train"][:3])["query"]
pos_tokenized = mmarco_collator(dataset["train"][:3])["positive"]
neg_tokenized = mmarco_collator(dataset["train"][:3])["negative"]
query_emb = model(**query_tokenized).last_hidden_state[:, 0]
pos_emb = model(**pos_tokenized).last_hidden_state[:, 0]
neg_emb = model(**neg_tokenized).last_hidden_state[:, 0]


In [173]:
pos_score = torch.bmm(query_emb.unsqueeze(1), torch.transpose(pos_emb.unsqueeze(1), 1, 2))[:, 0]
neg_score = torch.bmm(query_emb.unsqueeze(1), torch.transpose(neg_emb.unsqueeze(1), 1, 2))[:, 0]
pos_score, neg_score

(tensor([[202.0026],
         [133.8183],
         [143.7897]], grad_fn=<SelectBackward0>),
 tensor([[165.9211],
         [145.7220],
         [104.7045]], grad_fn=<SelectBackward0>))

In [171]:
torch.matmul(query_emb, torch.transpose(pos_emb, 0, 1))

tensor([[202.0026, 149.3506, 159.3111],
        [147.0003, 133.8183, 136.3080],
        [108.8749,  95.9988, 143.7897]], grad_fn=<MmBackward0>)

In [188]:
def sim(vec1, vec2):
    return -torch.bmm(vec1.unsqueeze(1), torch.transpose(vec2.unsqueeze(1), 1, 2))[:, 0]
loss_criterion = TripletMarginWithDistanceLoss(distance_function=sim)
loss_criterion(query_emb, pos_emb, neg_emb)

tensor(0., grad_fn=<MeanBackward0>)

In [1]:
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM, BertModel
model_name_or_path = "cl-tohoku/bert-base-japanese-v2"

/home/argon/anaconda3/envs/jpir/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class MMarcoCollator():
    def __init__(self, tokenizer, max_length=512):
        self.tokenizer = tokenizer
        self.max_length = max_length

    def _tokenize(self, texts):
        return self.tokenizer(texts,
                               padding=True,
                               truncation=True,
                               max_length=self.max_length,
                               return_tensors='pt')
    def _extract(self, key, examples):
        return list(map(lambda x: x[key], examples))
    
    def __call__(self, examples):
        encodings = {
            'query': self._tokenize(self._extract("query", examples)),
            'positive': self._tokenize(self._extract("positive", examples)),
            'negative': self._tokenize(self._extract("negative", examples)),
        }

        return encodings

tokenizer = BertJapaneseTokenizer.from_pretrained(model_name_or_path)
mmarco_collator = MMarcoCollator(tokenizer)

In [3]:
import torch.nn as nn
from transformers.modeling_outputs import ModelOutput
from torch.nn import TripletMarginWithDistanceLoss

class Encoder(nn.Module):
    def __init__(self, query_encoder, document_encoder, loss_function=None):
        super().__init__()
        self.query_encoder = query_encoder
        self.document_encoder = document_encoder
        self.loss_function = loss_function

    @classmethod
    def calc_similarity(cls, vectors1, vectors2):
        return torch.bmm(vectors1.unsqueeze(1), torch.transpose(vectors2.unsqueeze(1), 1, 2))[:, 0]

    def forward(self,
                query,
                positive=None,
                negative=None,
                attention_mask=None,
                position_ids=None,
                token_type_ids=None,
                output_attentions=False,
                output_hidden_states=False,):
        
        query_outputs = self.query_encoder(**query)
        query_emb = query_outputs.last_hidden_state[:, 0]

        if positive is None: raise
        positive_outputs = self.document_encoder(**positive)
        pos_emb = positive_outputs.last_hidden_state[:, 0]
        
        score = self.calc_similarity(query_emb, pos_emb)

        neg_emb = None
        if negative is not None:
            negative_outputs = self.document_encoder(**negative)
            neg_emb = negative_outputs.last_hidden_state[:, 0]

        loss = None
        if neg_emb is not None and self.loss_function is not None:
            loss = self.loss_function(query_emb, pos_emb, neg_emb)

        attentions=None
        if output_attentions:
            attentions=outputs.attentions
        
        hidden_states=None
        if output_hidden_states:
            hidden_states=outputs.hidden_states
        
        return ModelOutput(
            logits=score,
            loss=loss,
            last_hidden_state=query_outputs.last_hidden_state,
            attentions=attentions,
            hidden_states=hidden_states
        )

def distance(vec1, vec2):
    return -Encoder.calc_similarity(vec1, vec2)
loss_f = TripletMarginWithDistanceLoss(distance_function=distance)
q_encoder = BertModel.from_pretrained(model_name_or_path)
d_encoder = BertModel.from_pretrained(model_name_or_path)
net = Encoder(q_encoder, d_encoder, loss_f)

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertM

In [4]:
from datasets import load_dataset

dataset = load_dataset('unicamp-dl/mmarco', 'japanese')["train"]

seed = 42
test_rate = 0.2

train_size = int(len(dataset)*(1-test_rate))
test_size = int(len(dataset)*test_rate)

train_dataset = dataset.shuffle(seed=seed).select(range(train_size))
eval_dataset = dataset.shuffle(seed=seed).select(range(test_size))
train_dataset, eval_dataset

Found cached dataset mmarco (/home/argon/.cache/huggingface/datasets/unicamp-dl___mmarco/japanese/2.0.0/38e5159e2b13a7fdcd2af6415542b3c9a3bf27d70031abd9c4f814a2aa3367e2)
100%|██████████| 1/1 [00:00<00:00, 19.26it/s]
Loading cached shuffled indices for dataset at /home/argon/.cache/huggingface/datasets/unicamp-dl___mmarco/japanese/2.0.0/38e5159e2b13a7fdcd2af6415542b3c9a3bf27d70031abd9c4f814a2aa3367e2/cache-f078c07b33dd819f.arrow
Loading cached shuffled indices for dataset at /home/argon/.cache/huggingface/datasets/unicamp-dl___mmarco/japanese/2.0.0/38e5159e2b13a7fdcd2af6415542b3c9a3bf27d70031abd9c4f814a2aa3367e2/cache-f078c07b33dd819f.arrow


(Dataset({
     features: ['query', 'positive', 'negative'],
     num_rows: 31824648
 }),
 Dataset({
     features: ['query', 'positive', 'negative'],
     num_rows: 7956162
 }))

In [5]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./output/model',
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    lr_scheduler_type='constant',
    load_best_model_at_end=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    remove_unused_columns=False
)

In [6]:
from transformers import Trainer
from transformers import EarlyStoppingCallback

trainer = Trainer(
    model=net,
    tokenizer=tokenizer,
    data_collator=mmarco_collator,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train(ignore_keys_for_eval=['last_hidden_state', 'hidden_states', 'attentions'])

/home/argon/anaconda3/envs/jpir/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 31824648
  Num Epochs = 3
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 23868486


Epoch,Training Loss,Validation Loss


RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 6.00 GiB total capacity; 4.93 GiB already allocated; 0 bytes free; 5.31 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [18]:
import torch
import torch.nn.functional as F

# pos_sim = torch.tensor([[3.5], [1.5]]).squeeze(1)
# neg_sim = torch.tensor([[5.0], [1.4]]).squeeze(1)
pos_sim, neg_sim = torch.tensor([[760.6363],[75.3284]]), torch.tensor([[76.7528],[75.6246]])
res = torch.stack([pos_sim, neg_sim])
print("torch.stack([pos_sim, neg_sim]):", res)
res = F.log_softmax(res, dim=1)
res, res.shape, -1.0 * res[:, 0], (-1.0 * res[:, 0]).mean()

torch.stack([pos_sim, neg_sim]): tensor([[[760.6363],
         [ 75.3284]],

        [[ 76.7528],
         [ 75.6246]]])


(tensor([[[ 0.0000e+00],
          [-6.8531e+02]],
 
         [[-2.8037e-01],
          [-1.4086e+00]]]),
 torch.Size([2, 2, 1]),
 tensor([[-0.0000],
         [0.2804]]),
 tensor(0.1402))

In [2]:
[-1.2337e-04, -9.0001e+00, -3.0486e+00, -4.8587e-02]

[-0.00012337, -9.0001, -3.0486, -0.048587]

In [2]:
import torch
from model import BertDenseEncoder
from transformers import BertJapaneseTokenizer, BertModel
import torch.nn.functional as F

query = "情報検索"
pos_doc = "情報検索（じょうほうけんさく、英語: Information retrieval）とは、コンピュータを用いて大量のデータ群から目的に合致したものを取り 出すこと。"
neg_doc = "Microsoft の機械読解 (MRC) がテキストから意味をスキャンして抽出する方法について説明します。"

if isinstance(query, str): query = [query, query]
if isinstance(pos_doc, str): pos_doc = [pos_doc, pos_doc]
if isinstance(neg_doc, str): neg_doc = [neg_doc, neg_doc]

path = "cl-tohoku/bert-base-japanese-v2"
# path = "model/checkpoint-10000"
tokenizer = BertJapaneseTokenizer.from_pretrained(path)
tokenizer.add_tokens("[Q]", special_tokens=True)
tokenizer.add_tokens("[D]", special_tokens=True)

encoder = BertDenseEncoder.from_pretrained(path, tokenizer=tokenizer)

with torch.no_grad():
    max_length = 512
    query_tokenized = tokenizer(["[Q]"+text for text in query],
                            padding=True,
                            truncation=True,
                            max_length=max_length,
                            return_tensors='pt')
    pos_tokenized = tokenizer(["[D]"+text for text in pos_doc],
                            padding=True,
                            truncation=True,
                            max_length=max_length,
                            return_tensors='pt')
    neg_tokenized = tokenizer(["[D]"+text for text in neg_doc],
                            padding=True,
                            truncation=True,
                            max_length=max_length,
                            return_tensors='pt')
    # query_emb = encoder.get_emb(query_tokenized)
    # doc_emb = encoder.get_emb(document_tokenized)

    # output = encoder(query_tokenized, positive=document_tokenized)

    query_outputs = encoder.encoder(**query_tokenized)
    query_emb = query_outputs.last_hidden_state[:, 0]
    query_emb = encoder.linear(query_emb)

    positive_outputs = encoder.encoder(**pos_tokenized)
    pos_emb = positive_outputs.last_hidden_state[:, 0]
    pos_emb = encoder.linear(pos_emb)

    negative_outputs = encoder.encoder(**neg_tokenized)
    neg_emb = negative_outputs.last_hidden_state[:, 0]
    neg_emb = encoder.linear(neg_emb)

/home/argon/.pyenv/versions/anaconda3-2022.05/envs/jpir/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-v2 were not used when initializing BertDenseEncoder: ['bert.encoder.layer.10.attention.self.key.bias', 'bert.encoder.layer.9.attention.self.key.weight', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.7.output.dense.bias', 'bert.encoder.layer.11.attention.self.value.bias', 'bert.encoder.layer.2.intermediate.dense.bias', 'bert.encoder.layer.9.attention.self.key.bias', 'bert.encoder.layer.7.intermediate.dense.weight', 'bert.encoder.layer.1.intermediate.dense.weight', 'bert.encoder.layer.9.attention.self.value.weight', 'bert.encoder.layer.8.attention.self.query.bias', 'bert.encoder.layer.10.attention.output.dense.weig

In [4]:
# path = "cl-tohoku/bert-base-japanese-v2"
path = "model/checkpoint-15000"
tokenizer = BertJapaneseTokenizer.from_pretrained(path)
tokenizer.add_tokens("[Q]", special_tokens=True)
tokenizer.add_tokens("[D]", special_tokens=True)

encoder = BertDenseEncoder.from_pretrained(path, tokenizer=tokenizer)

from datasets import load_dataset
dataset = load_dataset('unicamp-dl/mmarco', 'japanese')["train"]

from run import MMarcoCollator

# print(mmarco_collator([dataset[0], dataset[1], dataset[2]]))
# return

seed = 42
# test_rate = 0.2
# train_size = int(len(dataset)*(1-test_rate))
# test_size = int(len(dataset)*test_rate)
train_size = 10000
test_size = 3000

train_dataset = dataset.shuffle(seed=seed).select([0,1])
mmarco_collator = MMarcoCollator(tokenizer)
train_dataset = mmarco_collator(train_dataset)

query, positive, negative = train_dataset["query"], train_dataset["positive"], train_dataset["negative"]

query_outputs = encoder.encoder(**query)
query_emb = query_outputs.last_hidden_state[:, 0]
query_emb = encoder.linear(query_emb)

assert positive is not None
positive_outputs = encoder.encoder(**positive)
pos_emb = positive_outputs.last_hidden_state[:, 0]
pos_emb = encoder.linear(pos_emb)

score = encoder.calc_similarity(query_emb, pos_emb)

neg_emb = None
if negative is not None:
    negative_outputs = encoder.encoder(**negative)
    neg_emb = negative_outputs.last_hidden_state[:, 0]
    neg_emb = encoder.linear(neg_emb)

print(pos_emb, neg_emb)
print(score)

Some weights of BertDenseEncoder were not initialized from the model checkpoint at model/checkpoint-15000 and are newly initialized: ['linear.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Found cached dataset mmarco (/home/argon/.cache/huggingface/datasets/unicamp-dl___mmarco/japanese/2.0.0/38e5159e2b13a7fdcd2af6415542b3c9a3bf27d70031abd9c4f814a2aa3367e2)
100%|██████████| 1/1 [00:00<00:00,  7.16it/s]


ModuleNotFoundError: No module named 'train'